### Getting ready

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import torchvision.models as models
import numpy as np
import random
import glob
import os
import cv2
import math
import torch.optim as optim
from transformers import MBartForConditionalGeneration, MBartTokenizer,MBartConfig
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-german-dbmdz-uncased')
from torchvision.models.video import r3d_18
import pickle

# Load the pre-trained model
i3d = r3d_18(pretrained=True).to("cuda")
i3d.eval()

c:\Users\Phoenix\Desktop\gflst\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Phoenix\Desktop\gflst\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Phoenix\Desktop\gflst\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VideoResNet(
  (stem): BasicStem(
    (0): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (conv2): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1):

In [ ]:
tokenizer.pad_token_id

0

In [ ]:
# Get the size of the tokenizer's vocabulary
vocab_size = len(tokenizer.vocab)
print("Vocabulary Size:", vocab_size)

Vocabulary Size: 31102


In [ ]:
seed = 10
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms(False)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Dataset

In [ ]:
train_corpus_path = r"PHOENIX-2014-T.train.corpus.csv"
dev_corpus_path = r"PHOENIX-2014-T.dev.corpus.csv"
test_corpus_path = r"PHOENIX-2014-T.test.corpus.csv"

train_videos_path = r"D:\phoenix-2014-T.v3\PHOENIX-2014-T-release-v3\PHOENIX-2014-T\features\fullFrame-210x260px\train"
dev_videos_path = r"D:\phoenix-2014-T.v3\PHOENIX-2014-T-release-v3\PHOENIX-2014-T\features\fullFrame-210x260px\dev"
test_videos_path = r"D:\phoenix-2014-T.v3\PHOENIX-2014-T-release-v3\PHOENIX-2014-T\features\fullFrame-210x260px\test"

['name', 'video', 'start', 'end', 'speaker', 'orth', 'translation']

In [ ]:
data_transform = transforms.Compose([
                                    transforms.ToTensor()
                                    ])

train_data = PHEONIX14T(train_corpus_path, train_videos_path, data_transform, 300,)
dev_data = PHEONIX14T(dev_corpus_path, dev_videos_path, data_transform, 300)
test_data = PHEONIX14T(test_corpus_path, test_videos_path, data_transform, 300)

In [ ]:
train_data.__getitem__(1)

(array([[[[0.83434874, 0.81956189, 0.83206517, ..., 0.83039216,
           0.8079066 , 0.84642857],
          [0.82731092, 0.80513065, 0.82280725, ..., 0.82959449,
           0.80705313, 0.83732493],
          [0.83606443, 0.81547072, 0.82193737, ..., 0.82865896,
           0.80807073, 0.83294818],
          ...,
          [0.87146359, 0.86505383, 0.86439076, ..., 0.88063725,
           0.85850293, 0.90042017],
          [0.78848039, 0.76065192, 0.74772847, ..., 0.85221573,
           0.83303571, 0.87923669],
          [0.71453081, 0.69206495, 0.67954854, ..., 0.82505252,
           0.80734638, 0.86008403]],
 
         [[0.86540616, 0.85061931, 0.87027202, ..., 0.86176471,
           0.83927915, 0.87780112],
          [0.8547619 , 0.83258163, 0.86349352, ..., 0.86096704,
           0.83842568, 0.86869748],
          [0.86509104, 0.84449733, 0.85888152, ..., 0.85845588,
           0.83786765, 0.8627451 ],
          ...,
          [0.86995798, 0.86354823, 0.87544096, ..., 0.91670168,
   

In [ ]:
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=2,
                              collate_fn=collate_fn,
                              shuffle=True,
                              worker_init_fn=seed_worker)

dev_dataloader = DataLoader(dataset=dev_data,
                             batch_size=2,
                             collate_fn=collate_fn,
                             shuffle=False,
                             worker_init_fn=seed_worker)

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=2,
                             collate_fn=collate_fn,
                             shuffle=False,
                             worker_init_fn=seed_worker)

In [ ]:
len(train_dataloader)

3548

In [ ]:
len(dev_dataloader)

260

### Model

In [ ]:
def extract_features(video_batch, window_size, stride, extractor, preprocess):
    features = []
    batch_size, max_len, H, W, C = video_batch.shape
    # Calculate number of clips (Tf) based on window size and stride
    Tf = (max_len - window_size) // stride + 1

    for i in range(batch_size):  # Loop over each video in the batch
        video_clips = []
        for j in range(0, max_len-window_size+1, stride):  # Loop over time dimension to create clips
            clip = video_batch[i, j:j+window_size]  # Get a clip
            clip = clip.permute(0, 3, 1, 2)  # Reshape clip to [T, C, H, W]
            clip = torch.stack([preprocess(frame) for frame in clip])  # Apply preprocessing to each frame
            video_clips.append(clip)

        # Convert list of tensors to a tensor
        video_clips = torch.stack(video_clips)

        # Pass each clip through the extractor
        clip_features = [extractor(clip) for clip in video_clips]
        # Combine features from all clips (stacking and reshaping as necessary)
        clip_features = torch.stack(clip_features)
        features.append(clip_features)

    # Combine features for the entire batch
    features = torch.stack(features)
    return features

## Decoder

In [ ]:
class TransformerDecoderLayer(nn.Module):
    """
    Transformer decoder layer.

    Consists of self-attention, source-attention, and feed-forward.
    """

    def __init__(
        self, size: int = 0, ff_size: int = 0, num_heads: int = 0, dropout: float = 0.1
    ):
        """
        Represents a single Transformer decoder layer.

        It attends to the source representation and the previous decoder states.

        :param size: model dimensionality
        :param ff_size: size of the feed-forward intermediate layer
        :param num_heads: number of heads
        :param dropout: dropout to apply to input
        """
        super(TransformerDecoderLayer, self).__init__()
        self.size = size

        self.trg_trg_att = nn.MultiheadAttention(size, num_heads,batch_first=True)
        self.src_trg_att = nn.MultiheadAttention(400, num_heads,batch_first=True)

        self.feed_forward = PositionWiseFeedForward(size, ff_size)
        # self.projection = nn.Linear(size, 400)
        self.x_layer_norm = nn.LayerNorm(size, eps=1e-6)
        self.dec_layer_norm = nn.LayerNorm(size, eps=1e-6)

        self.dropout = nn.Dropout(dropout)

    # pylint: disable=arguments-differ
    def forward(
        self,
        x,
        memory,
        src_mask,
        trg_mask,
    ):
        """
        Forward pass of a single Transformer decoder layer.

        :param x: inputs
        :param memory: source representations
        :param src_mask: source mask
        :param trg_mask: target mask (so as to not condition on future steps)
        :return: output tensor
        """
        # decoder/target self-attention
        x_norm = self.x_layer_norm(x)
        h1, scores1 = self.trg_trg_att(x_norm, x_norm, x_norm, attn_mask =trg_mask)

        h1 = self.dropout(h1) + x

        # source-target attention
        h1_norm = self.dec_layer_norm(h1)
        # h1_projected = self.projection(h1_norm)
        h2, scores2 = self.src_trg_att(memory, memory, h1_norm, src_mask)

        # final position-wise feed-forward layer
        o = self.feed_forward(self.dropout(h2) + h1)

        return o


In [ ]:
class TransformerDecoder(nn.Module):
    """
    A transformer decoder with N masked layers.
    Decoder layers are masked so that an attention head cannot see the future.
    """

    def __init__(
        self,
        num_layers: int = 3,
        num_heads: int = 8,
        hidden_size: int = 400,
        ff_size: int = 2048,
        dropout: float = 0.1,
        emb_dropout: float = 0.1,
        vocab_size: int = 31102,
        freeze: bool = False,
        **kwargs
    ):
        """
        Initialize a Transformer decoder.

        :param num_layers: number of Transformer layers
        :param num_heads: number of heads for each layer
        :param hidden_size: hidden size
        :param ff_size: position-wise feed-forward size
        :param dropout: dropout probability (1-keep)
        :param emb_dropout: dropout probability for embeddings
        :param vocab_size: size of the output vocabulary
        :param freeze: set to True keep all decoder parameters fixed
        :param kwargs:
        """
        super(TransformerDecoder, self).__init__()

        self._hidden_size = hidden_size
        self._output_size = vocab_size

        # create num_layers decoder layers and put them in a list
        self.layers = nn.ModuleList(
            [
                TransformerDecoderLayer(
                    size=hidden_size,
                    ff_size=ff_size,
                    num_heads=num_heads,
                    dropout=dropout,
                )
                for _ in range(num_layers)
            ]
        )

        self.pe = PositionalEncoding(hidden_size, 31102)
        self.layer_norm = nn.LayerNorm(hidden_size, eps=1e-6)

        self.emb_dropout = nn.Dropout(p=emb_dropout)
        self.output_layer = nn.Linear(hidden_size, vocab_size, bias=False)


    def forward(
        self,
        trg_embed= None,
        encoder_output= None,
        encoder_hidden= None,
        src_mask= None,
        unroll_steps: int = None,
        hidden= None,
        trg_mask= None,
        **kwargs
    ):
        """
        Transformer decoder forward pass.

        :param trg_embed: embedded targets
        :param encoder_output: source representations
        :param encoder_hidden: unused
        :param src_mask:
        :param unroll_steps: unused
        :param hidden: unused
        :param trg_mask: to mask out target paddings
                         Note that a subsequent mask is applied here.
        :param kwargs:
        :return:
        """
        assert trg_mask is not None, "trg_mask required for Transformer"

        x = self.pe(trg_embed)  # add position encoding to word embedding
        x = self.emb_dropout(x)

        # trg_mask = trg_mask & subsequent_mask(trg_embed.size(1)).type_as(trg_mask)

        for layer in self.layers:
            x = layer(x=x, memory=encoder_output, src_mask=src_mask, trg_mask=trg_mask)

        x = self.layer_norm(x)
        output = self.output_layer(x)

        return output, x, None, None

    def __repr__(self):
        return "%s(num_layers=%r, num_heads=%r)" % (
            self.__class__.__name__,
            len(self.layers),
            self.layers[0].trg_trg_att.num_heads,
        )

## Modules

In [ ]:
class InitializationModule(nn.Module):
    def __init__(self,
                hidden_size = 400,
                ff_size = 2048,
                num_layers = 3,
                num_heads = 8,
                dropout= 0.1):
        super(InitializationModule, self).__init__()
        self.encoder = TransformerEncoder()
        self.decoder = TransformerDecoder()

    def forward(self, video, translation,src_mask, tgt_mask):

        encoded = self.encoder(video, src_mask, initial = True)
        decoded = self.decoder(translation, encoded, src_mask=src_mask, trg_mask=tgt_mask)

        return encoded, decoded

In [ ]:
class IterativePrototypeRefinement(nn.Module):
    def __init__(self,
                hidden_size = 400,
                ff_size = 2048,
                num_layers = 3,
                num_heads = 8,
                dropout= 0.1,
                K = 3):
        super(IterativePrototypeRefinement, self).__init__()
        self.encoder = TransformerEncoder()
        self.decoder = TransformerDecoder()
        self.K = 3

    def forward(self, video, encoded_mem, translation,src_mask, tgt_mask):

        decoder_outputs = []
        for i in range(self.K):
            encoded = self.encoder(video, src_mask, encoded_mem)
            decoded = self.decoder(video, encoded, src_mask=src_mask, trg_mask=tgt_mask)
            decoder_outputs.append(decoded[0].detach().cpu())
        return decoder_outputs




In [ ]:
import torch

def create_look_ahead_mask(size):
    # Create a matrix where the entries above the diagonal are True (masked)
    mask = torch.triu(torch.ones((size, size)), diagonal=1).bool()
    return mask

def create_trg_mask(trg, pad_token_id):
    # trg: [batch_size, trg_len]
    # pad_token_id: the token ID used for padding (e.g., BERT's [PAD] token ID)

    # Create a padding mask for ignoring pad tokens
    pad_mask = (trg == pad_token_id).unsqueeze(1)  # Shape: [batch_size, 1, trg_len]

    # Create the look-ahead mask
    trg_len = trg.size(1)
    look_ahead_mask = create_look_ahead_mask(trg_len)  # Shape: [trg_len, trg_len]
    look_ahead_mask = look_ahead_mask.to(trg.device).expand(trg.size(0), trg_len, trg_len)  # [batch_size, trg_len, trg_len]

    # Combine the masks
    trg_mask = pad_mask | look_ahead_mask  # Shape: [batch_size, trg_len, trg_len]
    return trg_mask


In [ ]:
def get_batch_video_clips( video_batch, window_size, stride):
        batch_video_clips = []
        batch_size, max_len, H, W, C = video_batch.shape
        # Calculate number of clips (Tf) based on window size and stride
        Tf = (max_len - window_size) // stride + 1

        for i in range(batch_size):  # Loop over each video in the batch
            video_clips = []
            for j in range(0, max_len-window_size+1, stride):  # Loop over time dimension to create clips
                clip = video_batch[i, j:j+window_size]  # Get a clip
                clip = clip.permute(0, 3, 1, 2)  # Reshape clip to [T, C, H, W]
                video_clips.append(clip)

            # Convert list of tensors to a tensor
            video_clips = torch.stack(video_clips)
            batch_video_clips.append(video_clips)

        # Combine features for the entire batch
        batch_video_clips = torch.stack(batch_video_clips)
        return batch_video_clips

In [ ]:
class IPSLT(nn.Module):

    def __init__(self,
                hidden_size = 400,
                ff_size = 2048,
                num_layers = 3,
                num_heads = 8,
                dropout= 0.1, K = 3):
        super(IPSLT, self).__init__()
        self.initalization_module = InitializationModule()
        self.iterative_prototype_refinement = IterativePrototypeRefinement()
        # Parameters
        vocab_size = 1000  # Example vocabulary size
        embedding_dim = 400  # Number of features per embedding

        # Creating the embedding layer
        self.embedding_layer = nn.Embedding(31102, embedding_dim)



    def forward(self, batch_video_clips, translation):

        features_video_clips = []
        # Create a tensor of zeros with the same batch size (2) but with 25 additional features
        zeros_to_add = torch.zeros(translation.size(0), max(70 -translation.size(1), 0), dtype=torch.int64)

        # Concatenate the original tensor with the zeros tensor along the second dimension
        translation = torch.cat((translation, zeros_to_add), dim=1)
        embeddings = self.embedding_layer(translation.to(device))
        # with torch.no_grad():
        #     output = bert_model(translation)
        # embeddings = output.last_hidden_state  # (2, 42, 768)

        for clip_index in range(batch_video_clips.shape[1]):
            temp_batch_video_clips = batch_video_clips[:, clip_index].squeeze(1).permute(0, 2, 1, 3, 4).to(device)
            features = i3d(temp_batch_video_clips)
            features_video_clips.append(features.detach().cpu())

        features_video_clips =  torch.stack(features_video_clips, dim=1)
        required_shape = (2, 70, 400)
        num_new_rows = required_shape[1] - features_video_clips.shape[1]

        # Create a tensor of zeros with the appropriate shape
        zeros_to_add = torch.zeros(2, num_new_rows, 400)

        # Concatenate the original tensor with the zeros tensor
        features_video_clips = torch.cat((features_video_clips.to("cpu"), zeros_to_add), dim=1)
        sgn_mask = (features_video_clips != torch.zeros(400))[..., 0].to(device)#.unsqueeze(1)
        print(sgn_mask.shape)
        print(features_video_clips.shape)
        trg_mask = create_trg_mask(translation, 0).to(device)


        # # txt_input is used for teacher forcing, last one is cut off
        # txt_input = translation[:, :-1]
        # # txt is used for loss computation, shifted by one since BOS
        # txt = translation[:, 1:]
        # # we exclude the padded areas from the loss computation
        # txt_mask = (self.txt_input != 0).unsqueeze(1)
        # num_txt_tokens = (self.txt != 0).data.sum().item()
        features_video_clips = features_video_clips.to(device)

        decoder_outputs_new = []
        encoded, decoder_output = self.initalization_module(features_video_clips, embeddings, sgn_mask, trg_mask.repeat(8, 1, 1))
        decoder_outputs_new.append(decoder_output[0].cpu())
        decoder_outputs = self.iterative_prototype_refinement(features_video_clips, encoded, embeddings, sgn_mask, trg_mask.repeat(8, 1, 1))
        decoder_outputs_new.extend(decoder_outputs)
        features_video_clips = features_video_clips.detach().cpu()
        del features_video_clips
        return decoder_outputs_new

## Train

In [ ]:
def load_checkpoint(filepath, model, optimizer):
    """
    Load a model checkpoint and restore the model and optimizer states.

    Args:
    filepath (str): Path to the checkpoint file.
    model (torch.nn.Module): Model to load the state_dict into.
    optimizer (torch.optim.Optimizer): Optimizer to load the state_dict into.

    Returns:
    int: The epoch at which the checkpoint was saved.
    float: The loss at the checkpoint.
    """
    # Load the saved file
    checkpoint = torch.load(filepath)

    # Restore the model and optimizer state
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    # Return the epoch and loss from the checkpoint
    return checkpoint['epoch'], checkpoint['loss']


In [ ]:
model = IPSLT()
optimizer = optim.Adam(model.parameters(), lr=5e-5)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
epochs = 50


In [ ]:
# epoch, loss = load_checkpoint('best_model_checkpoint.pth', model, optimizer)
# print(f"Loaded checkpoint from epoch {epoch} with loss {loss}")

In [ ]:
import torch
import torch.nn.functional as F

def custom_loss_function(translation, decoder_outputs, K=3):

    logits = decoder_outputs[0].view(-1, decoder_outputs[0].shape[-1]).to("cpu")  # Shape: [batch_size * sequence_length, vocab_size]
    zeros_to_add = torch.zeros(translation.size(0), max(70 -translation.size(1), 0), dtype=torch.int64)
    translation = torch.cat((translation, zeros_to_add), dim=1)
    target_ids = translation.view(-1)

    token_ids = torch.argmax(decoder_outputs[0], dim=-1)

    # # Convert token IDs to sentences
    # sentences = []
    # for i in range(token_ids.size(0)):
    #     tokens = tokenizer.convert_ids_to_tokens(token_ids[i])
    #     sentence = tokenizer.convert_tokens_to_string(tokens)
    #     sentences.append(sentence)

    # # Print the sentences
    # for i, sentence in enumerate(sentences):
    #     print(f"Sentence {i+1}: {sentence}")

    # Calculate the cross entropy loss
    ce_loss = F.cross_entropy(logits, target_ids, ignore_index = 0)
    for decoder_output in decoder_outputs:

        ce_loss_2 = F.cross_entropy(decoder_output.view(-1, decoder_output.shape[-1]) , target_ids, ignore_index = 0)

    # Calculate the iterative distillation loss
    idl_loss = 0.0
    final_logits = decoder_outputs[-1].detach().view(-1, decoder_outputs[-1].shape[-1])  # Get the logits of the final output
    final_probs = F.softmax(final_logits, dim=1)  # Calculate the softmax probabilities of the final logits

    for i, decoder_output in enumerate(decoder_outputs[:-1]):  # Iterate over all but the last decoder output
        logits = decoder_output.view(-1, decoder_output.shape[-1])  # Flatten the output to [batch_size * sequence_length, vocab_size]
        probs = F.log_softmax(logits, dim=1)  # Compute log softmax probabilities
        idl_loss += F.kl_div(probs, final_probs, reduction='batchmean')  # Compute KL divergence

        # To ignore specific indices (e.g., index 0 for padding), you can apply a mask
        mask = (logits.argmax(dim=1) != 0)  # Create a mask where the maximum index is not 0
        idl_loss *= mask.float().mean()  # Apply the mask to the loss (adjust based on your specific needs)
     # Combine the losses
    combined_loss = ce_loss + 15 * idl_loss + ce_loss_2

    return combined_loss

In [ ]:
import torch
import torch.nn.functional as F

def custom_loss_function(translation, decoder_outputs, K=3):
    # Flatten logits and convert to CPU for loss computation
    logits = decoder_outputs[0].view(-1, decoder_outputs[0].shape[-1]).to("cpu")

    # Padding translation to match the maximum length (70)
    zeros_to_add = torch.zeros(translation.size(0), max(70 - translation.size(1), 0), dtype=torch.int64)
    translation = torch.cat((translation, zeros_to_add), dim=1)
    target_ids = translation.view(-1)

    # Calculate the cross-entropy loss for the first decoder output
    ce_loss = F.cross_entropy(logits, target_ids, ignore_index=0)

    # Calculate the cross-entropy loss for each decoder output and sum them up
    ce_loss_2 = 0.0
    for decoder_output in decoder_outputs:
        ce_loss_2 += F.cross_entropy(decoder_output.view(-1, decoder_output.shape[-1]), target_ids, ignore_index=0)

    # Calculate the iterative distillation loss (IDL)
    idl_loss = 0.0
    final_logits = decoder_outputs[-1].detach().view(-1, decoder_outputs[-1].shape[-1])
    final_probs = F.softmax(final_logits, dim=1)

    for i, decoder_output in enumerate(decoder_outputs[:-1]):
        logits = decoder_output.view(-1, decoder_output.shape[-1])
        log_probs = F.log_softmax(logits, dim=1)
        idl_loss += F.kl_div(log_probs, final_probs, reduction='batchmean')

        # Apply mask to ignore specific indices (e.g., index 0 for padding)
        mask = (logits.argmax(dim=1) != 0)
        idl_loss *= mask.float().mean()

    # Combine the losses with weights summing to 1
    weight_ce = 0.4
    weight_idl = 0.2
    weight_ce2 = 0.4

    combined_loss = weight_ce * ce_loss + weight_idl * idl_loss + weight_ce2 * ce_loss_2

    return combined_loss


In [ ]:
def save_checkpoint(state, filename="checkpoint.pth"):
    """
    Saves a model checkpoint.

    Args:
        state (dict): Contains model's state_dict, may include optimizer's state_dict
        filename (str): Path to save the checkpoint
    """
    torch.save(state, filename)
    print(f"Checkpoint saved to {filename}")


In [ ]:
from torch.utils.tensorboard import SummaryWriter

# Initialize TensorBoard writer
writer = SummaryWriter('runs/weight_2_2_1')

train_losses = []
dev_losses = []
best_val_loss = float('inf')
for epoch in range(0, epochs):
    model.train()  # Training mode
    total_train_loss = 0
    for batch, (imgs, translation) in enumerate(train_dataloader):
        batch_video_clips = get_batch_video_clips(imgs, 30, 20).to("cpu")
        translation = translation.to("cpu")

        optimizer.zero_grad()
        decoder_outputs = model(batch_video_clips, translation)
        loss = custom_loss_function(translation, decoder_outputs)
        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()

        # Log training loss every 100 batches
        if batch % 100 == 0:
            writer.add_scalar('Training loss', loss.item(), epoch * len(train_dataloader) + batch)
            print(f'Epoch [{epoch+1}/{epochs}], Batch Loss: {loss.item():.4f}')

    average_train_loss = total_train_loss / len(train_dataloader)
    train_losses.append(average_train_loss)
    writer.add_scalar('Average Training Loss', average_train_loss, epoch)
    print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {average_train_loss:.4f}')

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    total_val_loss = 0
    with torch.no_grad():
        for val_batch, (val_imgs, val_translation) in enumerate(dev_dataloader):
            if val_imgs.shape[1] < 30 or val_imgs.shape[0] < 2:
                continue
            batch_video_clips = get_batch_video_clips(val_imgs, 30, 20).to("cpu")
            val_translation = val_translation.to("cpu")
            val_decoder_outputs = model(batch_video_clips, val_translation)
            val_loss = custom_loss_function(val_translation, val_decoder_outputs)
            total_val_loss += val_loss.item()

    average_val_loss = total_val_loss / len(dev_dataloader)
    dev_losses.append(average_val_loss)
    writer.add_scalar('Average Validation Loss', average_val_loss, epoch)
    print(f'Epoch [{epoch+1}/{epochs}], Validation Loss: {average_val_loss:.4f}')

    # Save checkpoint if the validation loss improved
    if average_val_loss < best_val_loss:
        best_val_loss = average_val_loss
        save_checkpoint({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': best_val_loss,
        }, "best_model_checkpoint.pth")

print(f'Best Validation Loss: {best_val_loss:.4f}')
writer.close()


torch.Size([2, 70])
torch.Size([2, 70, 400])


c:\Users\Phoenix\Desktop\gflst\venv\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\cuda\CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch [1/50], Batch Loss: 21.0273
torch.Size([2, 70])
torch.Size([2, 70, 400])
torch.Size([2, 70])
torch.Size([2, 70, 400])
torch.Size([2, 70])
torch.Size([2, 70, 400])
torch.Size([2, 70])
torch.Size([2, 70, 400])
torch.Size([2, 70])
torch.Size([2, 70, 400])
torch.Size([2, 70])
torch.Size([2, 70, 400])


KeyboardInterrupt: 

In [ ]:
model.eval()  # Set the model to evaluation mode
total_val_loss = 0
with torch.no_grad():  # No gradients needed for validation, which saves memory and computations
    for val_batch, (val_imgs, val_translation) in enumerate(dev_dataloader):
        print(val_imgs.shape)
        if val_imgs.shape[1] < 30 or val_imgs.shape[0] <2:
            continue
        batch_video_clips = get_batch_video_clips(val_imgs, 30, 20).to("cpu")
        val_translation = val_translation.to("cpu")
        val_decoder_outputs = model(batch_video_clips, val_translation)
        val_loss = custom_loss_function(val_translation, val_decoder_outputs)
        total_val_loss += val_loss.item()

average_val_loss = total_val_loss / len(dev_dataloader)
dev_losses.append(average_val_loss)
print(f'Epoch [{epoch+1}/{epochs}], Validation Loss: {average_val_loss:.4f}')

torch.Size([2, 111, 224, 224, 3])
torch.Size([2, 148, 224, 224, 3])
torch.Size([2, 93, 224, 224, 3])
torch.Size([2, 165, 224, 224, 3])
torch.Size([2, 81, 224, 224, 3])
torch.Size([2, 103, 224, 224, 3])
torch.Size([2, 108, 224, 224, 3])
torch.Size([2, 105, 224, 224, 3])
torch.Size([2, 164, 224, 224, 3])
torch.Size([2, 116, 224, 224, 3])
torch.Size([2, 154, 224, 224, 3])
torch.Size([2, 126, 224, 224, 3])
torch.Size([2, 139, 224, 224, 3])
torch.Size([2, 117, 224, 224, 3])
torch.Size([2, 139, 224, 224, 3])
torch.Size([2, 114, 224, 224, 3])
torch.Size([2, 93, 224, 224, 3])
torch.Size([2, 62, 224, 224, 3])
torch.Size([2, 191, 224, 224, 3])
torch.Size([2, 173, 224, 224, 3])
torch.Size([2, 86, 224, 224, 3])
torch.Size([2, 228, 224, 224, 3])
torch.Size([2, 157, 224, 224, 3])
torch.Size([2, 97, 224, 224, 3])
torch.Size([2, 174, 224, 224, 3])
torch.Size([2, 99, 224, 224, 3])
torch.Size([2, 106, 224, 224, 3])
torch.Size([2, 167, 224, 224, 3])
torch.Size([2, 79, 224, 224, 3])
torch.Size([2, 58, 224

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1 but got size 2 for tensor number 1 in the list.

In [ ]:
# Validation phase
model.eval()  # Set the model to evaluation mode
total_val_loss = 0
with torch.no_grad():  # No gradients needed for validation, which saves memory and computations
    for val_batch, (val_imgs, val_translation) in enumerate(dev_dataloader):
        batch_video_clips = get_batch_video_clips(val_imgs, 20, 10) # [4, 5, 50, 3, 260, 210]
        batch_video_clips = batch_video_clips.to("cpu")
        val_translation = val_translation.to("cpu")
        print(batch_video_clips.shape)

        val_decoder_outputs = model(batch_video_clips, val_translation)
        val_translation = val_translation.to("cpu")
        val_loss = custom_loss_function(val_translation, val_decoder_outputs)
        del val_decoder_outputs

        total_val_loss += val_loss.item()

# Calculate average validation loss
average_val_loss = total_val_loss / len(dev_dataloader)
average_train_loss = total_train_loss / len(train_dataloader)
train_losses.append(average_train_loss)
dev_losses.append(average_val_loss)
print(f'Epoch [{epoch+1}/{epochs}], Validation Loss: {average_val_loss:.4f}')
print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {average_train_loss:.4f}')


torch.Size([2, 5, 30, 3, 224, 224])
Sentence 1: [CLS] das bedeutet viele wolken und immer wieder zum teil kra¤ftige schauer und gewitter [SEP]geborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeboren
Sentence 2: [CLS] tiefer luftdruck bestimmt in den na¤chsten tagen unser wetter [SEP]geborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeborengeboren
torch.Size([2, 6, 30, 

KeyboardInterrupt: 

In [ ]:
with open("loss_train", "wb") as fp:   #Pickling
    pickle.dump(train_losses, fp)